In [71]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib

In [72]:
autoencoder = load_model('/content/autoencoder_model.h5')
encoder_model = load_model('/content/encoder_model.h5')

In [73]:
# Compile the models manually
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
encoder_model.compile(optimizer='adam', loss='mean_squared_error')

In [74]:
df = pd.read_csv('SAML-D_cleaned_final.csv')

<ipython-input-74-434d0fa35970>:1: DtypeWarning: Columns (47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SAML-D_cleaned_final.csv')


In [75]:
df = df.drop(columns=['Unnamed: 0', 'risk_score'])

In [76]:
# Identify boolean columns and convert them to integers
bool_columns = df.select_dtypes(include=['bool', 'object']).columns

In [77]:
for col in bool_columns:
    # Convert boolean-like object columns to actual boolean type first
    if df[col].dtype == 'object':
        df[col] = df[col].map({'True': True, 'False': False, 'true': True, 'false': False})

    # Replace NaN values with a default value (e.g., False or 0)
    df[col].fillna(False, inplace=True)

    # Convert boolean columns to integers

    df[col] = df[col].astype(int)

In [78]:
if df.isnull().values.any():
    df = df.dropna()

In [79]:
# Selecting the X as everything except the target
X = df.drop(columns=['Is_Laundering'])
y = df['Is_Laundering']

In [80]:
#SCale the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [82]:
def calculate_risk_score(row):
    score = 0

    # High-Risk Countries
    if row['high_risk_countries']:
        score += 20  # Assigning a significant score

    # Transaction Amount
    amount_score = min(row['Amount'] / 1000, 20)  # Normalizing and capping at 20
    score += amount_score

    # Laundering Types (assuming specific feature names)
    laundering_types = [
        'Laundering_type_Behavioural_Change_1',
        'Laundering_type_Behavioural_Change_2',
        'Laundering_type_Bipartite',
        'Laundering_type_Cash_Withdrawal',
        'Laundering_type_Cycle',
        'Laundering_type_Deposit-Send',
        'Laundering_type_Fan_In',
        'Laundering_type_Fan_Out','Laundering_type_Gather-Scatter',
        'Laundering_type_Layered_Fan_In','Laundering_type_Layered_Fan_Out',
        'Laundering_type_Normal_Cash_Deposits','Laundering_type_Normal_Cash_Withdrawal',
        'Laundering_type_Normal_Fan_In','Laundering_type_Normal_Fan_Out',
        'Laundering_type_Normal_Foward','Laundering_type_Normal_Group',
        'Laundering_type_Normal_Mutual','Laundering_type_Normal_Periodical',
        'Laundering_type_Normal_Plus_Mutual',
       'Laundering_type_Normal_Small_Fan_Out',
       'Laundering_type_Normal_single_large', 'Laundering_type_Scatter-Gather',
       'Laundering_type_Single_large', 'Laundering_type_Smurfing',
       'Laundering_type_Stacked Bipartite', 'Laundering_type_Structuring',

        # Add more as per your model's expectations
    ]

    for laundering_type in laundering_types:
        if row[laundering_type]:
            # Adjust scoring logic based on each laundering type
            if laundering_type == 'Laundering_type_Smurfing':
                score += 15
            elif laundering_type == 'Laundering_type_Scatter-Gather':
                score += 15

    # Time of Transaction
    if row['Hour'] < 6 or row['Hour'] > 22:
        score += 10  # Transactions at odd hours get a higher score

    return score


In [83]:
# Function to process features and calculate risk score
def process_features_and_calculate_risk(features):
    # Convert features dictionary to DataFrame
    features_df = pd.DataFrame([features])

    # Normalize the features
    features_scaled = scaler.transform(features_df)

    # Pass through the encoder to get the reduced features
    reduced_features = encoder_model.predict(features_scaled)[0]

    # Calculate risk score
    risk_score = calculate_risk_score(features)

    return risk_score, reduced_features


In [95]:
features = {
    'Sender_account': 12345,
    'Receiver_account': 67890,
    'Amount': 1000,
    'Year': 2024,
    'Month': 7,
    'Day': 2,
    'Hour': 12,
    'Minute': 30,
    'Second': 15,
    'Payment_currency_Albanian lek':0,
    'Payment_currency_Dirham':1,
    'Payment_currency_Euro':0,
    'Payment_currency_Indian rupee':0,
    'Payment_currency_Mexican Peso':1,
    'Payment_currency_Moroccan dirham':0,
    'Payment_currency_Naira':0,
    'Payment_currency_Pakistani rupee':0,
    'Payment_currency_Swiss franc':0,
    'Payment_currency_Turkish lira':1,
    'Payment_currency_UK pounds':0,
    'Payment_currency_US dollar':0,
    'Payment_currency_Yen':0,
    'Received_currency_Albanian lek':0,
    'Received_currency_Dirham':0,
    'Received_currency_Euro':0,
    'Received_currency_Indian rupee':1,
    'Received_currency_Mexican Peso':0,
    'Received_currency_Moroccan dirham':0,
    'Received_currency_Naira':0,
    'Received_currency_Pakistani rupee':0,
    'Received_currency_Swiss franc':0,
    'Received_currency_Turkish lira':0,
    'Received_currency_UK pounds':0,
    'Received_currency_US dollar':0,
    'Received_currency_Yen':0,
    'Sender_bank_location_Albania':0,
    'Sender_bank_location_Austria':1,
    'Sender_bank_location_France':0,
    'Sender_bank_location_Germany':0,
    'Sender_bank_location_India':0,
    'Sender_bank_location_Italy':0,
    'Sender_bank_location_Japan':0,
    'Sender_bank_location_Mexico':0,
    'Sender_bank_location_Morocco':0,
    'Sender_bank_location_Netherlands':0,
    'Sender_bank_location_Nigeria':0,
    'Sender_bank_location_Pakistan':0,
    'Sender_bank_location_Spain':0,
    'Sender_bank_location_Switzerland':0,
    'Sender_bank_location_Turkey':0,
    'Sender_bank_location_UAE':0,
    'Sender_bank_location_UK':0,
    'Sender_bank_location_USA':0,
    'Receiver_bank_location_Albania':0,
    'Receiver_bank_location_Austria':0,
    'Receiver_bank_location_France':0,
    'Receiver_bank_location_Germany':0,
    'Receiver_bank_location_India':0,
    'Receiver_bank_location_Italy':0,
    'Receiver_bank_location_Japan':0,
    'Receiver_bank_location_Mexico':0,
    'Receiver_bank_location_Morocco':0,
    'Receiver_bank_location_Netherlands':0,
    'Receiver_bank_location_Nigeria':0,
    'Receiver_bank_location_Pakistan':0,
    'Receiver_bank_location_Spain':0,
    'Receiver_bank_location_Switzerland':0,
    'Receiver_bank_location_Turkey':0,
    'Receiver_bank_location_UAE':0,
    'Receiver_bank_location_UK':0,
    'Receiver_bank_location_USA':0,
    'Payment_type_ACH':0,
    'Payment_type_Cash Deposit':0,
    'Payment_type_Cash Withdrawal':0,
    'Payment_type_Cheque':0,
    'Payment_type_Credit card':0,
    'Payment_type_Cross-border':1,
    'Payment_type_Debit card':0,
    'Laundering_type_Behavioural_Change_1':0,
    'Laundering_type_Behavioural_Change_2':1,
    'Laundering_type_Bipartite':1,
    'Laundering_type_Cash_Withdrawal':0,
    'Laundering_type_Cycle':0,
    'Laundering_type_Deposit-Send':0,
    'Laundering_type_Fan_In':0,
    'Laundering_type_Fan_Out':0,
    'Laundering_type_Gather-Scatter':0,
    'Laundering_type_Layered_Fan_In':0,
    'Laundering_type_Layered_Fan_Out':0,
    'Laundering_type_Normal_Cash_Deposits':0,
    'Laundering_type_Normal_Cash_Withdrawal':0,
    'Laundering_type_Normal_Fan_In':1,
    'Laundering_type_Normal_Fan_Out':0,
    'Laundering_type_Normal_Foward':0,
    'Laundering_type_Normal_Group':0,
    'Laundering_type_Normal_Mutual':0,
    'Laundering_type_Normal_Periodical':0,
    'Laundering_type_Normal_Plus_Mutual':0,
    'Laundering_type_Normal_Small_Fan_Out':0,
    'Laundering_type_Normal_single_large':0,
    'Laundering_type_Scatter-Gather':0,
    'Laundering_type_Single_large':0,
    'Laundering_type_Smurfing':0,
    'Laundering_type_Stacked Bipartite':0,
    'Laundering_type_Structuring':0,         # Replace with actual values as per your dataset
    'high_risk_countries': 0
}


In [96]:
risk_score, reduced_features = process_features_and_calculate_risk(features)
print("Risk Score:", risk_score)
print("Reduced Features:", reduced_features)

1/1 [==============================] - 0s 121ms/step
Risk Score: 1.0
Reduced Features: [ 4.993172   5.790073  14.401524   3.2185137  0.3269629  4.0952272
  0.         4.858855 ]
